In [1]:
import numpy as np
%matplotlib inline

これまで我々が見てきたNNは，フィードフォワードと呼ばれるネットワークで，流れが1方向でしかなかった.  
フィードフォワードでは時系列データをうまく扱うことができない．  
そこでリカレントニューラルネットワーク(RNN)の出番である．  
本章では，フィードフォワードの問題点を指摘し，RNNがその問題を解決することを示す．

# 確率と言語モデル

## word2vecを確率の視点から眺める

コーパス$w_1, w_2, ... w_T$が与えられているとき，$w_t$がターゲットとなる確率は，コンテキスト$w_{t-1}, w{t+1}$を使って
$$ P(w_t | w_{t-1}, w_{t+1})$$
と書ける．  
ここで，コンテキストの窓を非対称にして全て左側にコンテキストがあるとすると，$w_t$がターゲットとなる確率は
$$ P(w_t | w_{t-2}, w_{t-1})$$
このとき，CBOWモデルが扱う損失関数は
$$ L=-\log P(w_t|w_{t-2}, w_{t-1}) $$

## 言語モデル
言語モデルは，単語の並びがどれだけ自然であるかを確率で評価する．  
例えば，
- you say goodbye -> 0.092
- you say good die -> 0.000000000000032  

$ w_1, ..., w_m $ という順序で単語が出現する確率は，同時確率 $ P(w_1, ..., w_m) $ で表される．  
これを事後確率と確率の乗法定理 $P(A, B) = P(A|B)P(B)$ を使って分解すると　　
$$ P(w_1, ..., w_m) = P(w_m|w_1, ... w_{m-1})P(w_{m-1}|w_1, ... w_{m-2}) ... P(w_3|w_1, w_2)P(w_2|w_1)P(w_1)$$  
$$ = \prod_{t=1}^{m} P(w_t|w_1, ..., w_{t-1})$$

確率の乗法定理は，「AとBが両方同時に起こる確率 $P(A,B)$」は，「Bが起こる確率$P(B)$」と「Bが起こったあとにAが起きる確率P(A|B)」を掛け合わせたものである． 
また，$P(A, B) = P(B|A)P(A)$と書くこともできる．  
ここで注目すべきは，事後確率が対象の単語より左の全ての単語をコンテキストとした時の確率ということである．  
また， $P(w_t|w_1, ... w_{t-1})$を表すモデルは，条件付き言語モデルと呼ばれる．これを言語モデルと呼ぶ場合も多く見られる．

## CBOWモデルを言語モデルに？
CBOWモデルを無理やり言語モデルに適用するには，コンテキストのサイズをある値に限定することで近似的に表すことができる．  
$$ P(w_1, ..., w_m) = \prod_{t=1}^{m} P(w_t|w_1, ..., w_{t-1}) \approx \prod_{t=1}^{m} P(w_t|w_{t-2}, w_{t-1}) $$
  
マルコフ性  
未来の状態が現在の状態だけに依存して決まること．  
ここで，ある事象の確率がその直前のN個の事象だけに依存するとき，これを「N階マルコフ連鎖」という． 　
今は直前の2つに依存して次の単語が決まるので，2階マルコフ連鎖と呼べる．  
  
コンテキストのサイズは任意に設定できるが，固定する必要があるところに問題がある．  
例えば，コンテキストのサイズが10だが答えとなるTomなどの固有名詞が18個前にしかない時，この推論問題に答えることはできない．  
コンテキストを20にしたりすれば答えることはできる．  
  
しかし次にはコンテキスト内の単語の並びが無視されるという問題がある．  
例えば，(you,say)というコンテキストと(say,you)というコンテキストが同じものとして表される．  
これは，CBOWモデルの中間層を各コンテキストが共有しているために起きる．  
そこで，コンテキストごとに中間層を設けることで，すなわち複数の中間層を「連結(contcatenate)」することでこの問題に対処できる．  
しかし，そのようにするとコンテキストのサイズに比例して重みパラメータが増大してしまう．  

これらの問題を解決するのがRNNである．  
RNNは，コンテキストがどれだけ長くても，そのコンテキストの情報を記憶するメカニズムを持つ．  
  
ちなみに，実はword2vecの方が後に提案されている．  
RNNによる言語モデルでも単語の分散表現を獲得できるのだが，新たな語彙の追加しやすさや単語の分散表現の質の向上のためword2vecが提案された．

# RNNとは
Recurrent Neural Networkは日本語では「再起ニューラルネットワーク」や「循環ニューラルネットワーク」と呼ばれる．  
これに対してRecursive Neural Networkというものもあるが，こちらは主に木構造のデータを処理するためのネットワークで，RNNとは別物である．

## 循環するニューラルネットワーク
RNNの特徴は，閉路を持つことである．  
入力データを$(x_0, x_1, ... x_t, ...)$として， 出力データ$(h_0, h_1, ... h_t, ...)$ を自分にも入力する，すなわち閉路を持つ層をRNNレイヤと名付ける．　　
ここで，　$x_t$や$h_t$はベクトルを想定する．例えば，ある単語の分散表現を$x_t$としたりする．  
また，これまではデータが左から右へ流れていたが，以降は左右方向にレイヤが展開されるため，紙面の都合上，下から上へデータが流れるように描画される．  

## ループの展開
RNNレイヤは自身に出力データを流していたが，これを同じレイヤの別のRNNレイヤに流すことで，ループを展開する．  
左から右へ，同じレイヤのRNNレイヤが並び，その順が時系列の順番になっている．　　
その出力が，左から$h_0, h_1, ... h_t$となる．  
